In [1]:
!pip install underthesea bs4 requests

Defaulting to user installation because normal site-packages is not writeable


In [20]:
import requests
from bs4 import BeautifulSoup
import os

In [5]:
url = "https://vanbanphapluat.co/api/search?lv=/linh-vuc/bao-hiem&p="
uids_baohiem = {}
for i in range(1, 50):
    url = url + str(i)
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        items = data.get("Items", [])

        list_uid = {item.get("Title"):item.get("UID") for item in items if 'het-hieu-luc' not in item.get("TrinhTrangHieuLuc").get("UID")}
        # Extract UIDs from each item
        uids_baohiem.update(list_uid)
    else:
        print("Error fetching data from the API")
    url = url.strip(str(i))

In [6]:
url = "https://vanbanphapluat.co/api/search?lv=/linh-vuc/thue-phi-le-phi&p="
uids_thue = {}
for i in range(1, 50):
    url = url + str(i)
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        items = data.get("Items", [])

        list_uid = {item.get("Title"):item.get("UID") for item in items for item in items if 'het-hieu-luc' not in item.get("TrinhTrangHieuLuc").get("UID")}
        # Extract UIDs from each item
        uids_thue.update(list_uid)
    else:
        print("Error fetching data from the API")
    url = url.strip(str(i))

In [12]:
url = "https://vanbanphapluat.co/api/search?l=/loai-van-ban/luat&lv=/linh-vuc/thue-phi-le-phi&p="
uids_luat_thue = {}
for i in range(1, 4):
    url = url + str(i)
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        items = data.get("Items", [])
        list_uid = {item.get("Title"):item.get("UID") for item in items for item in items if 'het-hieu-luc' not in item.get("TrinhTrangHieuLuc").get("UID")}
        # Extract UIDs from each item
        uids_luat_thue.update(list_uid)
    else:
        print("Error fetching data from the API")
    url = url.strip(str(i))

In [15]:
url = "https://vanbanphapluat.co/api/search?l=/loai-van-ban/luat&lv=/linh-vuc/bao-hiem&p="
uids_luat_baohiem = {}
url = url + str(1)
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    items = data.get("Items", [])

    list_uid = {item.get("Title"):item.get("UID") for item in items for item in items if 'het-hieu-luc' not in item.get("TrinhTrangHieuLuc").get("UID")}
    # Extract UIDs from each item
    uids_luat_baohiem.update(list_uid)
else:
    print("Error fetching data from the API")

In [16]:
print (uids_luat_baohiem)

{'Luật Bảo hiểm xã hội': '/luat-bao-hiem-xa-hoi', 'Luật Kinh doanh bảo hiểm 2022': '/luat-kinh-doanh-bao-hiem-sua-doi', 'Luật Bảo hiểm y tế': '/luat-bao-hiem-y-te', 'Law 42/2019/QH14 amendments of Law on Insurance Business and Law on Intellectual Property': '/law-42-2019-qh14-amendments-of-law-on-insurance-business-and-law-on-intellectual-property', 'Luật Kinh doanh bảo hiểm Luật Sở hữu trí tuệ sửa đổi 2019': '/luat-sua-doi-luat-kinh-doanh-bao-hiem-luat-so-huu-tri-tue', 'Luật Bảo hiểm xã hội 2014': '/luat-bao-hiem-xa-hoi-2014', 'Luật Bảo hiểm y tế sửa đổi 2014': '/luat-bao-hiem-y-te-sua-doi-2014', 'Luật việc làm năm 2013': '/luat-viec-lam-nam-2013', 'Luật Bảo hiểm tiền gửi 2012': '/luat-bao-hiem-tien-gui-2012', 'Luật kinh doanh bảo hiểm sửa đổi 2010': '/luat-kinh-doanh-bao-hiem-sua-doi-2010', 'Luật bảo hiểm y tế 2008 25/2008/QH12': '/luat-bao-hiem-y-te-2008-25-2008-qh12', 'Luật Kinh doanh bảo hiểm 2000 24/2000/QH10': '/luat-kinh-doanh-bao-hiem-2000-24-2000-qh10'}


In [17]:
uids = dict(uids_baohiem)
uids.update(uids_thue)
uids.update(uids_luat_thue)
uids.update(uids_luat_baohiem)
print (len(uids))

1972


In [18]:
def html_to_text(html_string):
    # Remove HTML tags
    soup = BeautifulSoup(html_string, 'html.parser')
    for table_elem in soup.find_all('table'):
        table_elem.decompose()
    for center_elem in soup.find_all(attrs={'align': 'center'}):
        center_elem.decompose()
    for text_center_elem in soup.find_all(style=lambda value: value and 'text-align: center' in value):
        text_center_elem.decompose()
    for elem in soup.find_all(['p', 'br']):
        elem.replace_with(elem.text + '[BREAK]')
    plain_text = soup.get_text()
    return plain_text

In [21]:
url = "https://vanbanphapluat.co/api/vanban"
old_idx = -1
for title, uid in uids.items():
    if old_idx != -1:
        url = url +  uid + url[old_idx + len(uid):]
    else:
        url = url +  uid
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        html_content = data.get("ToanVan")
        plain_text = html_to_text(html_content)
        break_contained_texts = [text for text in plain_text.split() if "[BREAK]" != text]
        plain_text = " ".join(break_contained_texts)
        plain_text = plain_text.replace('[BREAK]', '\n')
        plain_text = plain_text.strip(" ")
        filename = "/mnt/d/bt/luan/data_files2/" + title.replace('/', '-') + ".txt"
        if os.path.exists(filename):
            continue
        else:
            with open(filename, 'w') as text_file:
                text_file.write(plain_text)
    else:
        print("Error fetching data from the API")
    old_idx = url.rfind(uid)
    url = url[:old_idx]